In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
print("TensorFlow version:", tf.__version__)
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
import shutil

TensorFlow version: 2.11.0


In [2]:
 %reload_ext tensorboard
from tensorflow.keras import Model
from pathlib import Path
import pandas as pd

In [3]:
BATCH_SIZE = 64
BUFFER_SIZE = 128
LEARNING_RATE = 0.05
EPOCHS = 50

In [4]:
input_dir = Path('./data/prepared')
logs_path = Path('./data/logs')
if logs_path.exists():
  shutil.rmtree(logs_path)
logs_path.mkdir(parents=True)

X_train_name = input_dir / 'X_train.csv'
y_train_name = input_dir / 'y_train.csv'
X_test_name = input_dir / 'X_test.csv'
y_test_name = input_dir / 'y_test.csv'

X_train = pd.read_csv(X_train_name)
y_train = pd.read_csv(y_train_name)
X_test = pd.read_csv(X_test_name)
y_test = pd.read_csv(y_test_name)

train_ds = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [5]:
X_train.shape

(436, 8)

In [6]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 8), dtype=tf.int64, name=None), TensorSpec(shape=(None, 1), dtype=tf.float64, name=None))>

In [14]:
class SomeModel(Model):
  def __init__(self, hidden_layers=3, neurons_cnt=128):
    super(SomeModel, self).__init__()
    self.hidden_layers = hidden_layers

    self.d_in = Dense(8, activation='relu')
    for cnt in range(self.hidden_layers):
      self.__dict__[f'd{cnt}'] = Dense(neurons_cnt, activation='relu')
    self.d_out = Dense(1)

  def call(self, x):
    x = self.d_in(x)
    for cnt in range(self.hidden_layers):
      x = self.getattr(f'd{cnt}')(x)
    return self.d2(x)

In [15]:
# Create an instance of the model
model = SomeModel()

In [17]:
loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE)
getattr(model, 'd2')

In [16]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.MeanAbsoluteError(name='train_mae')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.MeanAbsoluteError(name='test_mae')

In [17]:
@tf.function
def train_step(input_vector, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(input_vector, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [18]:
@tf.function
def test_step(input_vector, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(input_vector, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [19]:
from tensorflow import keras

In [20]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = logs_path / 'gradient_tape' / current_time / 'train'
train_log_dir.mkdir(exist_ok=True, parents=True)
test_log_dir = logs_path / 'gradient_tape' / current_time / 'test'
test_log_dir.mkdir(exist_ok=True, parents=True)
train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
test_summary_writer = tf.summary.create_file_writer(str(test_log_dir))

logdir=logs_path / "fit" / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir.mkdir(exist_ok=True, parents=True)
fit_summary_writer = tf.summary.create_file_writer(str(logdir))

tf.summary.trace_on(graph=True, profiler=True)
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  for (x_train, y_train) in train_ds:

    with fit_summary_writer.as_default():
      train_step(x_train, y_train)


  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_ds:
    test_step(x_test, y_test)

  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('mae', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test MAE: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result(),
                         test_loss.result(),
                         test_accuracy.result()))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

with fit_summary_writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

Instructions for updating:
use `tf.profiler.experimental.start` instead.
Epoch 1, Loss: 110.58494567871094, Accuracy: 9.363460540771484, Test Loss: 2.1504695415496826, Test MAE: 1.2225948572158813
Epoch 2, Loss: 0.9907283186912537, Accuracy: 0.7904427647590637, Test Loss: 0.7129884958267212, Test MAE: 0.6529427170753479
Epoch 3, Loss: 0.5725390315055847, Accuracy: 0.5754972696304321, Test Loss: 0.8268589973449707, Test MAE: 0.5893129706382751
Epoch 4, Loss: 0.6476150751113892, Accuracy: 0.6240902543067932, Test Loss: 0.8105593323707581, Test MAE: 0.5840927362442017
Epoch 5, Loss: 0.647804319858551, Accuracy: 0.6016566157341003, Test Loss: 1.1471900939941406, Test MAE: 0.7461292743682861
Epoch 6, Loss: 0.685904860496521, Accuracy: 0.6145110726356506, Test Loss: 0.6916478872299194, Test MAE: 0.5871939659118652
Epoch 7, Loss: 0.5783920288085938, Accuracy: 0.5796703100204468, Test Loss: 0.7662931084632874, Test MAE: 0.572924017906189
Epoch 8, Loss: 0.7019361853599548, Accuracy: 0.645323872

In [21]:
model.save('./data/models/mymodel')

INFO:tensorflow:Assets written to: ./data/models/mymodel\assets


In [22]:
loaded_model = keras.models.load_model('./data/models/mymodel')

In [25]:
loaded_model

In [24]:
model

In [26]:
# Let's check:
import numpy as np
np.testing.assert_allclose(
    model.predict(X_test), loaded_model.predict(X_test)
)
%tensorboard --logdir ./data/logs

4/4 [==============================] - 0s 5ms/step


Launching TensorBoard...

In [19]:
%tensorboard --logdir ./data/logs/gradient_tape

Launching TensorBoard...